<a href="https://colab.research.google.com/github/oduoranto/Machine-Learning/blob/main/Rice_Prices_Prediction_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [364]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler, StandardScaler, PolynomialFeatures
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn import metrics
import datetime

In [365]:
food_prices = pd.read_csv('/content/wfp_food_prices_ken (1).csv')

In [366]:
food_prices.head()

,date,admin1,admin2,market,latitude,longitude,category,commodity,unit,priceflag,pricetype,currency,price,usdprice
0,#date,#adm1+name,#adm2+name,#loc+market+name,#geo+lat,#geo+lon,#item+type,#item+name,#item+unit,#item+price+flag,#item+price+type,#currency,#value,#value+usd
1,2006-01-15,Coast,Mombasa,Mombasa,-4.05,39.666667,cereals and tubers,Maize,KG,actual,Wholesale,KES,16.13,0.2235
2,2006-01-15,Coast,Mombasa,Mombasa,-4.05,39.666667,cereals and tubers,Maize (white),90 KG,actual,Wholesale,KES,1480.0,20.5041
3,2006-01-15,Coast,Mombasa,Mombasa,-4.05,39.666667,pulses and nuts,Beans (dry),90 KG,actual,Wholesale,KES,3246.0,44.9705
4,2006-01-15,Eastern,Kitui,Kitui,-1.366667,38.016667,cereals and tubers,Potatoes (Irish),50 KG,actual,Wholesale,KES,1249.99,17.3175


In [367]:
food_prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10648 entries, 0 to 10647
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   date       10648 non-null  object
 1   admin1     10648 non-null  object
 2   admin2     10648 non-null  object
 3   market     10648 non-null  object
 4   latitude   10648 non-null  object
 5   longitude  10648 non-null  object
 6   category   10648 non-null  object
 7   commodity  10648 non-null  object
 8   unit       10648 non-null  object
 9   priceflag  10648 non-null  object
 10  pricetype  10648 non-null  object
 11  currency   10648 non-null  object
 12  price      10648 non-null  object
 13  usdprice   10648 non-null  object
dtypes: object(14)
memory usage: 1.1+ MB


In [368]:
food_prices.columns

Index(['date', 'admin1', 'admin2', 'market', 'latitude', 'longitude',
       'category', 'commodity', 'unit', 'priceflag', 'pricetype', 'currency',
       'price', 'usdprice'],
      dtype='object')

In [369]:
print(food_prices['commodity'].unique()) if 'commodity' in food_prices.columns else print("'commodity' column not found in the DataFrame.")

['#item+name' 'Maize' 'Maize (white)' 'Beans (dry)' 'Potatoes (Irish)'
 'Sorghum' 'Milk (cow, pasteurized)' 'Beans' 'Bread' 'Oil (vegetable)'
 'Fuel (diesel)' 'Fuel (kerosene)' 'Fuel (petrol-gasoline)' 'Maize flour'
 'Rice' 'Wheat flour' 'Milk (UHT)' 'Cooking fat' 'Bananas' 'Onions (red)'
 'Tomatoes' 'Potatoes (Irish, red)' 'Beans (kidney)' 'Beans (rosecoco)'
 'Beans (yellow)' 'Cabbage' 'Potatoes (Irish, white)' 'Kale'
 'Rice (aromatic)' 'Sorghum (red)' 'Cowpeas' 'Onions (dry)' 'Spinach'
 'Maize (white, dry)' 'Millet (finger)' 'Fish (omena, dry)'
 'Cowpea leaves' 'Sorghum (white)' 'Beans (dolichos)' 'Beans (mung)'
 'Rice (imported, Pakistan)' 'Salt' 'Sugar' 'Meat (beef)' 'Meat (goat)'
 'Milk (camel, fresh)' 'Milk (cow, fresh)' 'Meat (camel)']


In [370]:
rice_variations = ['Rice', 'Rice (aromatic)', 'Rice (imported, Pakistan)']
rice_data = food_prices[food_prices['commodity'].isin(rice_variations)]
print(rice_data.head)

<bound method NDFrame.head of              date         admin1   admin2                  market  \
4198   2020-08-15        Nairobi  Nairobi                 Nairobi   
4280   2021-01-15        Eastern    Kitui      Kitui town (Kitui)   
4314   2021-01-15  North Eastern  Garissa  Garissa town (Garissa)   
4321   2021-01-15  North Eastern  Mandera        Takaba (Mandera)   
4340   2021-01-15    Rift Valley   Nakuru       Wakulima (Nakuru)   
...           ...            ...      ...                     ...   
10597  2024-03-15    Rift Valley  Turkana       Ethiopia (Kakuma)   
10610  2024-03-15    Rift Valley  Turkana                Kakuma 2   
10617  2024-03-15    Rift Valley  Turkana                Kakuma 3   
10628  2024-03-15    Rift Valley  Turkana   Kalobeyei (Village 1)   
10633  2024-03-15    Rift Valley  Turkana   Kalobeyei (Village 2)   

               latitude         longitude            category  \
4198          -1.283333         36.816667  cereals and tubers   
4280       

In [371]:
rice_data.columns

Index(['date', 'admin1', 'admin2', 'market', 'latitude', 'longitude',
       'category', 'commodity', 'unit', 'priceflag', 'pricetype', 'currency',
       'price', 'usdprice'],
      dtype='object')

In [372]:
rice_data.shape

(452, 14)

In [373]:
rice_data.isnull().sum()

,0
date,0
admin1,0
admin2,0
market,0
latitude,0
longitude,0
category,0
commodity,0
unit,0
priceflag,0


In [374]:
rice_data.head()

,date,admin1,admin2,market,latitude,longitude,category,commodity,unit,priceflag,pricetype,currency,price,usdprice
4198,2020-08-15,Nairobi,Nairobi,Nairobi,-1.283333,36.816667,cereals and tubers,Rice,KG,actual,Retail,KES,104.0,0.9634
4280,2021-01-15,Eastern,Kitui,Kitui town (Kitui),-1.35657,38.00825,cereals and tubers,Rice (aromatic),50 KG,actual,Wholesale,KES,5625.0,51.1364
4314,2021-01-15,North Eastern,Garissa,Garissa town (Garissa),-0.459682,39.64136,cereals and tubers,Rice (aromatic),50 KG,actual,Wholesale,KES,5750.0,52.2727
4321,2021-01-15,North Eastern,Mandera,Takaba (Mandera),3.394616,40.225882,cereals and tubers,Rice (aromatic),50 KG,actual,Wholesale,KES,6400.0,58.1818
4340,2021-01-15,Rift Valley,Nakuru,Wakulima (Nakuru),-0.303099,36.080026,cereals and tubers,"Rice (imported, Pakistan)",50 KG,actual,Wholesale,KES,4670.0,42.4545


In [375]:
rice_data['priceflag'].value_counts()
rice_data['category'].value_counts()
rice_data['pricetype'].value_counts()
rice_data['admin1'].value_counts()


,count
admin1,
Rift Valley,206
North Eastern,100
Nairobi,83
Eastern,25
Coast,25
Nyanza,10
Central,3


In [376]:
#Changing the words to numeric values
rice_data.replace({'commodity': {'Rice': 0, 'Rice (aromatic)': 1, 'Rice (imported, Pakistan)': 2}}, inplace=True)
rice_data.replace({'unit': {'50 KG': 50, 'KG': 0}}, inplace=True)
rice_data.replace({'priceflag': {'aggregate': 0, 'actual': 1}}, inplace = True)
rice_data.replace({'pricetype': {'Wholesale': 0, 'Retail': 1}}, inplace = True)
rice_data.replace({'admin1': {'Rift Valley': 0, 'North Eastern':1, 'Nairobi': 3, 'Eastern': 4, 'Coast': 5, 'Nyanza': 6, 'Central': 7  }}, inplace=True)



<ipython-input-376-f0a8a83cceed>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  rice_data.replace({'commodity': {'Rice': 0, 'Rice (aromatic)': 1, 'Rice (imported, Pakistan)': 2}}, inplace=True)
<ipython-input-376-f0a8a83cceed>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rice_data.replace({'commodity': {'Rice': 0, 'Rice (aromatic)': 1, 'Rice (imported, Pakistan)': 2}}, inplace=True)
<ipython-input-376-f0a8a83cceed>:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future v

Splitting training and test data

In [391]:
rice_data['date'] = pd.to_datetime(rice_data['date']).apply(lambda x: x.toordinal())
rice_data.head()
rice_data.isnull().sum()
rice_data.shape


<ipython-input-391-6ec2cc3a1110>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rice_data['date'] = pd.to_datetime(rice_data['date']).apply(lambda x: x.toordinal())


(452, 14)

In [378]:
X = rice_data.drop(['price','currency','admin2', 'market', 'category','usdprice'], axis=1)
Y = rice_data['price']

Splitting the data into training and test data

In [379]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=2)
X_train.head()

,date,admin1,latitude,longitude,commodity,unit,priceflag,pricetype
6004,738321,0,0.516667,35.283333,1,50,1,0
6752,738655,0,-0.303099,36.080026,1,50,1,0
8433,738017,1,-0.459682,39.64136,0,0,0,1
9899,738655,1,0.110807,40.316079,0,0,0,1
9249,738139,0,3.752637,34.823365,0,0,0,1


Model Training

In [380]:
linear_model = LinearRegression()
linear_model.fit(X_train, Y_train)

LinearRegression()

Model evaluation

In [381]:
test_data_prediction1 = linear_model.predict(X_test)
error_score = metrics.r2_score(Y_test, test_data_prediction1)
print( 'R score = ', error_score)

R score =  0.9291595818087315


Random Forest Regressor

In [382]:
regressor = RandomForestRegressor(n_estimators=100, random_state=0)
regressor.fit(X_train, Y_train)

RandomForestRegressor(random_state=0)

In [383]:
regressor_prediction = regressor.predict(X_test)
error_score = metrics.r2_score(Y_test, regressor_prediction)
print( 'R score = ', error_score)

R score =  0.9663225990856268


Decison Tree regressor

In [384]:
decisonTree = DecisionTreeRegressor()
decisonTree.fit(X_train, Y_train)

DecisionTreeRegressor()

In [385]:
tree_prediction = regressor.predict(X_test)
error_score = metrics.r2_score(Y_test, tree_prediction)
print( 'R score = ', error_score)

R score =  0.9663225990856268


Lasso regression

In [386]:
lasso_regressor = Lasso()
lasso_regressor.fit(X_train, Y_train)

Lasso()

In [387]:
lasso_regressor_prediction = regressor.predict(X_test)
error_score = metrics.r2_score(Y_test, lasso_regressor_prediction)
print( 'R score = ', error_score)

R score =  0.9663225990856268


In [388]:
new_data = pd.DataFrame({
    'date': [datetime.date(2020, 1, 15)],
    'admin1': [1],
    'latitude': [3.394616],
    'longitude': [40.225882],
    'commodity': [1],
    'unit': [50],
    'priceflag': [1],
    'pricetype': [0]
})

# Convert 'date' to ordinal time (matching how your model was trained)
new_data['date'] = new_data['date'].apply(lambda x: x.toordinal())

prediction = lasso_regressor.predict(new_data)
print('The price = ', prediction)

The price =  [6006.84134667]
